# generate_capsid_mvsj.ipynb
- 指定したT-numberのcapsidのPDB情報を[VIPERdb](https://viperdb.org/)から取得し、[MolViewSpec](https://molstar.org/mol-view-spec/)を使ってmvsj形式で書き出すためのノートブックです。
- mvsjファイルは、[molstar](https://molstar.org/)という分子ビューアで読み込むことができます。

## 注意
事前準備として、SSL証明書を取得しておく必要があります。
1. [viperdb]()にアクセスして、サーバーの証明書・中間証明書・ルート証明書をエクスポート
2. 証明書を連結する
   ```
   cat [サーバの証明書] [中間証明書] [ルート証明書] > full_chain.pem
   ```

上の作業が面倒な場合は、自己責任で以下のコードの`cert=False`に設定してください。

## T=1だけmvsjにエクスポート

In [ ]:
T_NUMBER = 1 # capsidのT-number
BASE_URL = 'http://viperdb.org' # VIPERdbのURL
BASE_DIR = 'mvsj_files' # ファイルを保存するディレクトリ
os.makedirs(BASE_DIR, exist_ok=True)
CERTIFICATE = './full_chain.pem' # SSL証明書

In [7]:
import molviewspec as mvs
import os
import numpy as np
import requests
from tqdm.notebook import tqdm
from matplotlib import colormaps as cmaps

In [8]:
def get_chain_colors(num_chains, cmap='tab20'):
    """
    Get a list of colors for each chain in a PDB entry.
    Parameters
    ----------
    num_chains : int
        The number of chains in the PDB entry.
    cmap : str, optional
        The name of the matplotlib colormap to use for generating colors.
    Returns
    -------
    chain_colors : list
        A list of color codes for each chain in the PDB entry.
    """
    # カラーマップの取得
    colormap = cmaps.get_cmap(cmap)
    # チェーン数に応じたカラーの取得
    chain_colors = [colormap(i%len(colormap.colors)) for i in range(num_chains)]
    # 16進数のカラーコードに変換
    chain_colors = [f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}' for r, g, b, _ in chain_colors]
    return chain_colors

# test
get_chain_colors(21)

['#1f77b4',
 '#aec7e8',
 '#ff7f0e',
 '#ffbb78',
 '#2ca02c',
 '#98df8a',
 '#d62728',
 '#ff9896',
 '#9467bd',
 '#c5b0d5',
 '#8c564b',
 '#c49c94',
 '#e377c2',
 '#f7b6d2',
 '#7f7f7f',
 '#c7c7c7',
 '#bcbd22',
 '#dbdb8d',
 '#17becf',
 '#9edae5',
 '#1f77b4']

In [9]:
def get_chain_info(pdb_id):
    """
    Get the number of chains and chain instances for a PDB entry.
    Parameters
    ----------
    pdb_id : str
        The PDB ID of the entry.
    Returns
    -------
    chain_count : int
        The number of chains in the PDB entry.
    chain_instances : list
        The chain instances in the PDB entry.
    """
    url = f'https://data.rcsb.org/rest/v1/core/polymer_entity/{pdb_id}/1'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # チェーン数の取得
        chain_count = len(data.get('rcsb_polymer_entity_container_identifiers', {}).get('auth_asym_ids', []))
        # 各チェーンのインスタンス数の取得
        chain_instances = data.get('rcsb_polymer_entity_container_identifiers', {}).get('auth_asym_ids', [])
        return chain_count, chain_instances
    else:
        # print(f'Error: Unable to fetch data for PDB ID {pdb_id}')
        return None, None


In [ ]:
def save_mvsj(pdb_id, filepath, cmap='tab20'):
    """
    Save a MolViewSpec JSON file for a PDB entry.
    Parameters
    ----------
    pdb_id : str
        The PDB ID of the entry to save.
    """
    # call `create_builder()` to create an instance of the builder
    builder = mvs.create_builder()
    # at each step, auto-complete will suggest possible actions depending on the current state of the builder
    structure = builder.download(url=f"https://files.wwpdb.org/download/{pdb_id.lower()}.cif").parse(format="mmcif").assembly_structure()

    chain_count, chain_instances = get_chain_info(pdb_id)

    # PDBが存在しない場合は、処理を終了(2stvなど)
    if chain_count is None:
        return
    
    if chain_count != 0:
        chain_colors = get_chain_colors(chain_count, cmap=cmap)
        for i, chain in enumerate(chain_instances):
            # 各chainについて、surfaceを描画
            # print(f"Chain {chain}: {chain_instances[i]} instances")
            chain_structure = structure.component(selector=mvs.ComponentExpression(auth_asym_id=chain))
            chain_structure.representation(type="surface").color(color=chain_colors[i])
    else:
        # chainがない場合は、全体をsurfaceで描画
        structure.component().representation(type="surface").color(color="white")

    # finally, we pretty-print everything to the console
    # print(builder.get_state())
    builder.save_state(destination=filepath)

In [12]:
response = requests.get(f"{BASE_URL}/services/tnumber_index.php?serviceName=familiesAndtnumbers&tnumber={T_NUMBER}", verify=CERTIFICATE)
if response.status_code == 200:
    entries = response.json()
    print("Entries:", entries)
    print("Number of Families:", len(entries))
    print("Number of Entries:", np.sum([len(entries[family]) for family in entries]))
else:
    print("Failed to fetch Entries:", response.status_code)

Entries: {'Adenoviridae': [{'entry_id': '2c9g', 'name': 'Adenovirus Type 2 Penton Base Dodecahedron', 'family': 'Adenoviridae', 'genus': 'Mastadenovirus', 'resolution': '9.30'}, {'entry_id': '2c9f', 'name': 'Adenovirus Type 3 Penton', 'family': 'Adenoviridae', 'genus': 'Mastadenovirus', 'resolution': '16.50'}, {'entry_id': '4aqq', 'name': 'DODECAHEDRON FORMED OF PENTON BASE PROTEIN FROM ADENOVIRUS A', 'family': 'Adenoviridae', 'genus': 'Mastadenovirus', 'resolution': '4.75'}, {'entry_id': '4ar2', 'name': 'DODECAHEDRON FORMED OF PENTON BASE PROTEIN FROM ADENOVIRUS A', 'family': 'Adenoviridae', 'genus': 'Mastadenovirus', 'resolution': '3.80'}, {'entry_id': '1x9p', 'name': 'Human Adenovirus 2 Penton Base', 'family': 'Adenoviridae', 'genus': 'Mastadenovirus', 'resolution': '3.30'}, {'entry_id': '1x9t', 'name': 'Human Adenovirus 2 Penton Base In Complex With An Ad2 N-Terminal Fibre Peptide', 'family': 'Adenoviridae', 'genus': 'Mastadenovirus', 'resolution': '3.50'}, {'entry_id': '2c6s', 'na

In [13]:
for family in tqdm(entries.items(), leave=False):
    # print()
    family_name = family[0]
    # print(f"### Family: {family}")
    # print(f"Number of Entries: {len(family[1])}")
    # os.makedirs(f"{BASE_DIR}/{T_NUMBER}/{family_name}", exist_ok=True)
    entries_in_family = family[1]

    if type(entries_in_family) == dict:
        entries_in_family = list(entries_in_family.values())

    for entry in tqdm(entries_in_family, leave=False):
        # print(f"Entry: {entry}")
        # print(f"genus: {entry['genus']}")
        # print(f"entry_id: {entry['entry_id']}")
        entry_id = entry['entry_id']
        genus = entry['genus']
        os.makedirs(f"{BASE_DIR}/T_{T_NUMBER}/{family_name}/{genus}", exist_ok=True)
        save_mvsj(entry_id, f"{BASE_DIR}/T_{T_NUMBER}/{family_name}/{genus}/{entry_id}.mvsj")
        
    

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]